# Necessary Libraries



*   This notebook requires GPU to make it work properly. Do not forget to select T4 GPU in the environment settings before starting.



In [ ]:
!pip install "langchain[docarray]"
!pip install langchain-community
!pip install 'pydantic==1.10.9'
!pip install 'gradio==3.48.0'
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install sentence-transformers

# Read review - Q&A data

In [7]:
import pandas as pd
review_qa_df = pd.read_csv("/content/review_qa_sample_data.csv")
review_qa_df.head(5)

,review_qa_data
0,kutuda şarj cihazı mevcut mudur USB var efendim
1,hocam ortalama fiyati ne olur geldiginde? 400 ...
2,Merhaba. Bugün mi band 5 siparişi verdim de gü...
3,Merhabalar saatin ekranı kırılırsa tam olarak ...
4,garanti için girusebioecegimiz telefon numaras...


# Initialize Trendyol LLM

In [ ]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import pandas as pd
import torch
import random
import numpy as np

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

# fix randomness for reproducibility
set_seed()
model_id = "Trendyol/Trendyol-LLM-7b-chat-dpo-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    load_in_8bit=True
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    max_new_tokens=1024, # adjust model's output text size
    return_full_text=True,
    no_repeat_ngram_size=3, # prevent repetitive texts during text generation
    temperature=0.1, # adjust this parameter to stabilize model outputs
    do_sample=True
)

llm = HuggingFacePipeline(pipeline=pipe)

# Initialize local vector database

In [ ]:
from langchain.chains import RetrievalQA
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate

# get embedder
embedder = HuggingFaceEmbeddings(
    model_name="paraphrase-multilingual-mpnet-base-v2"
)

SCORE_THRESH = 0.1
TOP_K = 3

# generate local vector db
local_vector_db = DocArrayInMemorySearch.from_texts(
    review_qa_df["review_qa_data"].tolist(), embedder
)
retriever = local_vector_db.as_retriever(
    search_kwargs={"score_threshold": SCORE_THRESH, "k": TOP_K}
)

# Initialize Conversational RAG Chain

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts.chat import SystemMessagePromptTemplate, ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
import langchain

prompt_template = """ [INST] <<SYS>>
    Sen Trendyol'da çalışan ve ürünlerle ilgili sorulara cevap veren bir asistansın.
    <</SYS>>
    Sana verilen müşteri sorusunu, '''ÜRÜN YORUMLARI VE SORU - CEVAPLARI''' başlığı altında verilen bilgileri kullanarak cevapla.
    Cevapların Türkçe olmalı.
    Cevapların özenli bir üsluba sahip olmalı.
    Cevapların özet ve doğru bilgiler içermeli.
    Ürün adı: ""Xiaomi Mi Band 5 Akıllı Bileklik siyah""

    '''ÜRÜN YORUMLARI VE SORU & CEVAPLARI''':
    - {context} [/INST]

    Sohbet geçmişini ve takip sorusunu bağımsız bir soruda birleştir. Gerekirse cevabını destekleyecek bir açıklama ekle.
    Sohbet geçmişi: {chat_history}
    Takip sorusu: {question}
"""

condense_question_prompt = PromptTemplate(
    input_variables=["context", "chat_history", "question", "product_name"],
    template=prompt_template
)

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=False,
    k=3 # adjust to track number of chat turns to consider as memory
)
convrag_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    get_chat_history=lambda h: h,
    verbose=False,
    chain_type="stuff",
    rephrase_question=False,
    combine_docs_chain_kwargs={'prompt': condense_question_prompt}
)

# Launch chatbot & interact

In [ ]:
import gradio as gr

def interact(query, history):
    return convrag_chain.invoke(query)["answer"].strip().replace("Cevap", "").replace(":", "")

demo = gr.ChatInterface(
    fn=interact,
    examples=[
        "Uyku takip özelliği var mı?",
        "Kardeşime almak mantıklı mı sence? Teknolojik ürünlerle arası iyi ama hiç akıllı bileklik kullanmadı."
    ],
    title="AI Shopping Assistant",
    description="This app is using Trendyol-LLM-7b-chat-dpo-v1.0 model as a shopping assistant.\n Currently, you can ask your questions for the 'Xiaomi Mi Band 5 Akıllı Bileklik Siyah' product."
)

demo.launch(share=True)